In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
calendar=pd.read_csv('../input/airbnb-rome/calendar.csv')
listings=pd.read_csv('../input/airbnb-rome/listings.csv')

**Será feito Inner Join entre as tabelas "calendar" e "listings".**

In [ ]:
calendar_exp = listings.merge(calendar, how='inner',left_on=['id','price','minimum_nights','maximum_nights'], right_on=['listing_id','price','minimum_nights','maximum_nights'] )

In [ ]:
calendar_exp.info()

In [ ]:
calendar_exp.tail()

In [ ]:
 calendar_exp['listing_id'].equals(calendar_exp['id'])

**Testado positivamente que a colunas "calendar_exp['listing_id']" é idêntica à coluna "calendar_exp['id']"**

In [ ]:
 calendar_exp.drop(columns=['listing_id'], inplace = True)

In [ ]:
! pip install pycaret

In [ ]:
from pycaret.regression import *

**Vamos aplicar o setup do pycaret regression, visando obter o modelo de previsão da coluna price**

In [ ]:
calendar_exp['price'] = calendar_exp['price'].str.slice(start=1)
calendar_exp['price'] = calendar_exp['price'].str.slice(start=0,stop=-3)
calendar_exp['price'] = calendar_exp['price'].str.replace("[^0-9]", "", n=-1, case=None, regex=True)
calendar_exp['price'] = pd.to_numeric(calendar_exp['price'], errors='raise')

In [ ]:
calendar_exp['price'] = calendar_exp['price'].astype('int')

In [ ]:
print(calendar_exp['price'].dtype)

In [ ]:
calendar_exp['price'].describe()

In [ ]:
pd.unique(calendar_exp['price'])

**Abaixo foram especificadass as colunas que se referem à dados categóricos, dados numéricos e dados que pderiam ser descartados para ana´lise do modelo de aprendizado de máquina.**

In [ ]:
numeric_features1 = ['accommodates','minimum_nights','maximum_nights','availability_30','availability_60','availability_90','availability_365','number_of_reviews','number_of_reviews_ltm','number_of_reviews_l30d','calculated_host_listings_count','calculated_host_listings_count_entire_homes','calculated_host_listings_count_private_rooms','calculated_host_listings_count_shared_rooms']

In [ ]:
numeric_features2 = ['host_listings_count','host_total_listings_count','bathrooms','bedrooms','beds','minimum_minimum_nights','maximum_minimum_nights','minimum_maximum_nights','maximum_maximum_nights','minimum_nights_avg_ntm','maximum_nights_avg_ntm','review_scores_rating','review_scores_accuracy','review_scores_cleanliness','review_scores_checkin','review_scores_communication','review_scores_location','review_scores_value','reviews_per_month','latitude','longitude']

In [ ]:
numeric_features = numeric_features1 + numeric_features2

In [ ]:
numeric_features

In [ ]:
categorical_features = ['id','scrape_id','host_id','listing_url' ,'last_scraped','description','neighborhood_overview','picture_url','host_url','host_name','host_location','host_is_superhost','neighbourhood_cleansed','property_type','room_type','instant_bookable','available']

In [ ]:
ignore_features=['name','listing_url','last_scraped','description','neighborhood_overview','picture_url','host_url','host_name','host_since','host_location','host_about','host_response_time','host_response_rate','host_acceptance_rate','host_thumbnail_url','host_picture_url','host_neighbourhood','host_verifications','neighbourhood','calendar_last_scraped','first_review','last_review','license','adjusted_price','host_has_profile_pic','host_identity_verified','neighbourhood_group_cleansed','bathrooms_text','amenities','has_availability','instant_bookable']

In [ ]:
date_features=['date','host_since']

**Na linha de código logo abaixo foi criada uma coluna para facilitar o binnings dos 'id' em 10 folds, e assim aplicar a estratégia groupkfold**

In [ ]:
calendar_exp['decil_id'] = pd.cut(calendar_exp['id'], bins=10, labels=['1 decil id', '2 decil id', '3 decil id', '4 decil id','5 decil id','6 decil id','7 decil id','8 decil id','9 decil id','10 decil id'])

In [ ]:
calendar_exp_jump = calendar_exp[::500]

**De início, conseguiu-se executar o comando setup com coletando as 250.000 primeiras linhas da tabela 'calendar_exp'. Então, como a tabela tem 4.185.140 linhas, optou-se por rodar novamente o comanda setup para coletar de 17 em 17 linhas. Como invariavelmente surgia 'crash' de memória,optou-se por executar de 500 em 500 linhas.**

In [ ]:
reg = setup(data=calendar_exp_jump,target="price",ignore_features=ignore_features,fold_strategy='groupkfold',fold_groups='decil_id',silent=False,categorical_features=categorical_features,numeric_features=numeric_features,date_features=date_features)

**O comando "compare_models()" do pycaret demorou muto tempo (até 9 horas) para executar. Por decurso de o prazo o mesmo foi abortado, sem testar todas as estratégias de de machine learning**.

In [ ]:
random_forest = create_model('rf')

In [ ]:
evaluate_model(random_forest)

In [ ]:
Light_Gradient_Boosting_Machine = create_model('lightgbm')

In [ ]:
evaluate_model(Light_Gradient_Boosting_Machine)

In [ ]:
tuned_Light_Gradient_Boosting_Machine = tune_model(Light_Gradient_Boosting_Machine, n_iter = 10)

In [ ]:
evaluate_model(tuned_Light_Gradient_Boosting_Machine)

In [ ]:
lin_reg = create_model('lr')

In [ ]:
evaluate_model(lin_reg)

In [ ]:
predict_model(random_forest)

In [ ]:
predict_model(lin_reg)

In [ ]:
predict_model(tuned_Light_Gradient_Boosting_Machine)

In [ ]:
predict_model(Light_Gradient_Boosting_Machine)

**O modelo de aprendizado de máquina de maior R2 foi o LGBM. Portanto, foi aplicado o coando finalize model, para aumentar mais ainda o R2.**

In [ ]:
final_Light_Gradient_Boosting_Machine = finalize_model(Light_Gradient_Boosting_Machine)

In [ ]:
predict_model(final_Light_Gradient_Boosting_Machine)

In [ ]:
final_random_forest = finalize_model(random_forest)

In [ ]:
predict_model(final_random_forest)

In [ ]:
treino1 = calendar_exp[::1900]

In [ ]:
predict_model(final_Light_Gradient_Boosting_Machine,data=treino1)

In [ ]:
treino2 = calendar_exp[::2300]

In [ ]:
predict_model(final_Light_Gradient_Boosting_Machine,data=treino2)

In [ ]:
predict_model(final_random_forest,data=treino1)

In [ ]:
predict_model(final_random_forest,data=treino2)

In [ ]:
evaluate_model(final_Light_Gradient_Boosting_Machine)

In [ ]:
evaluate_model(final_random_forest)

In [ ]:
save_model(final_Light_Gradient_Boosting_Machine,'LGBM_cqw9')

In [ ]:
save_LGBM = save_model(final_Light_Gradient_Boosting_Machine,'LGBM_cqw9')

In [ ]:
save_model(final_random_forest,'Random_Forest_cqw9')

In [ ]:
save_Random_Forest = save_model(final_random_forest,'Random_Forest_cqw9')